<div style="background-color: steelblue; padding: 10px; border-radius: 5px;">
    <p style="margin: 10;"></p>
    <h1 style="text-align: center; margin: 0; font-weight: bold; color: white;">SWOT Ocean Model Intercomparison Project : EORCA12-ERA5v1 Test Case</h1>
    <p style="margin: 10;"></p>
</div>



## 📦**Imports** 

In [1]:
%%time

##################################
#Imports

from datetime import datetime
import s3fs
import xarray as xr
import pyinterp
from widetrax import DataPreprocessing as dp
from widetrax import Spectra as sp
import numpy as np
import json
from watermark import watermark
import time
import platform

CPU times: user 3.62 s, sys: 610 ms, total: 4.23 s
Wall time: 1.69 s


## 🔍 **Check the S3 Endpoint**  

In [2]:
s3_folder ="s3://project-moi-swot-omip/GLO36V1" # Do not write /!!!!!!
endpoint_url =  "https://minio.dive.edito.eu/"
fs = s3fs.S3FileSystem(anon=True,endpoint_url=endpoint_url)
# List contents of the bucket
bucket_name = 's3://project-moi-swot-omip/GLO36V1'
contents = fs.ls(bucket_name)
print("Bucket contents:")
for item in contents:
    print(item)

Bucket contents:
project-moi-swot-omip/GLO36V1/cycle_.keep
project-moi-swot-omip/GLO36V1/cycle_008
project-moi-swot-omip/GLO36V1/cycle_009
project-moi-swot-omip/GLO36V1/cycle_010
project-moi-swot-omip/GLO36V1/cycle_011
project-moi-swot-omip/GLO36V1/cycle_012
project-moi-swot-omip/GLO36V1/cycle_013
project-moi-swot-omip/GLO36V1/cycle_014
project-moi-swot-omip/GLO36V1/cycle_015
project-moi-swot-omip/GLO36V1/cycle_016
project-moi-swot-omip/GLO36V1/cycle_017
project-moi-swot-omip/GLO36V1/cycle_018
project-moi-swot-omip/GLO36V1/cycle_019
project-moi-swot-omip/GLO36V1/cycle_020
project-moi-swot-omip/GLO36V1/cycle_021
project-moi-swot-omip/GLO36V1/cycle_022
project-moi-swot-omip/GLO36V1/cycle_023
project-moi-swot-omip/GLO36V1/cycle_024
project-moi-swot-omip/GLO36V1/cycle_025
project-moi-swot-omip/GLO36V1/cycle_026


## 🧮 **Required variables**

In [3]:
# REGION A

name_region="Region_B"
season="JFM"

if name_region=="Region_A":
    lon_min = 360-70,
    lon_max = 360-60,
    lat_min = 32,
    lat_max = 42,
elif name_region=="Region_B":
    lon_min = 360-50,
    lon_max = 360-40,
    lat_min = 17,
    lat_max = 27,

area = [lon_min, lat_min, lon_max, lat_max]

if season=="JFM":    
    start_date = "01012024" # "DDMMYYYY"
    end_date ="31032024"
elif season=="JAS":
    start_date = "01072024" # "DDMMYYYY"
    end_date ="30092024"

In [4]:
end_date

'31032024'

## 🔄 **Identify the cycle numbers within the specified time range** 

In [5]:
if season =="JFM":
    file_path = "https://minio.lab.dive.edito.eu/project-meom-ige/cycles_periods.csv" # works only for winter period
elif season =="JAS":
    file_path = "time_ranges.csv"  # for summer

matching_cycles = dp.get_matching_cycles(file_path, start_date, end_date)

def formater_numeros_concis(liste_numeros):
  return [str(numero).zfill(3) for numero in liste_numeros]
    
matching_cycles = formater_numeros_concis(matching_cycles)
matching_cycles


['008', '009', '010', '011', '012', '013']

## 📊 **Load datasets and compute Wavenumber spectra**

In [6]:
%%time

# Load datasets into a dictionary
datasets_dict = dp.read_swot_ncfiles_S3subfolders(
    s3_folder,
    matching_cycles,
    endpoint_url,
    area)

08_505_ included.
08_533_ included.
08_546_ included.
08_561_ included.
08_574_ included.
09_005_ included.
09_018_ included.
09_033_ included.
09_046_ included.
09_074_ included.
09_102_ included.
09_199_ included.
09_227_ included.
09_255_ included.
09_268_ included.
09_283_ included.
09_296_ included.
09_311_ included.
09_324_ included.
09_352_ included.
09_380_ included.
09_505_ included.
09_533_ included.
09_546_ included.
09_561_ included.
09_574_ included.
10_005_ included.
10_018_ included.
10_033_ included.
10_046_ included.
10_074_ included.
10_102_ included.
10_199_ included.
10_227_ included.
10_255_ included.
10_268_ included.
10_283_ included.
10_296_ included.
10_311_ included.
10_324_ included.
10_352_ included.
10_380_ included.
10_505_ included.
10_533_ included.
10_546_ included.
10_561_ included.
10_574_ included.
11_005_ included.
11_018_ included.
11_033_ included.
11_046_ included.
11_074_ included.
11_102_ included.
11_199_ included.
11_227_ included.
11_255_ in

In [7]:
%%time 
has_converged, filled_datasets = dp.fill_nan(datasets_dict, varname = "ssh")

CPU times: user 3.08 s, sys: 6.92 s, total: 10 s
Wall time: 4.29 s


In [8]:
%%time
segments_dict = sp.retrieve_segments(filled_datasets,FileType = "NetCDF",namevar="ssh")

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/widetrax/Spectra.py:43: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for col in range(dataset.dims['num_pixels']):


CPU times: user 268 ms, sys: 4.37 ms, total: 273 ms
Wall time: 271 ms


In [9]:
%%time
psd_dict, freqs_dict = sp.calculate_psd(segments_dict)
# Calculate PSD Mean
psd_mean, freqs_mean = sp.psd_mean_and_freq(psd_dict,freqs_dict)

CPU times: user 745 ms, sys: 501 ms, total: 1.25 s
Wall time: 4.81 s


In [11]:
psd_mean

array([5.86788547e-02, 1.95864747e+00, 3.24991576e-01, 2.09067938e-01,
       1.24619461e-01, 7.82027910e-02, 4.38615983e-02, 3.68700502e-02,
       2.59036747e-02, 1.73075587e-02, 1.04160043e-02, 7.28847391e-03,
       5.30196995e-03, 4.40389201e-03, 3.51661285e-03, 3.24634471e-03,
       2.85842636e-03, 2.50621370e-03, 2.15839174e-03, 1.76178495e-03,
       1.16319187e-03, 1.04970524e-03, 8.94301526e-04, 7.15003875e-04,
       6.33847228e-04, 5.63264826e-04, 4.86636498e-04, 4.36743816e-04,
       3.84836802e-04, 3.16769885e-04, 2.62643784e-04, 2.14454446e-04,
       1.99000126e-04, 1.65749082e-04, 1.46644374e-04, 1.41314824e-04,
       1.32078779e-04, 1.16701950e-04, 1.07317475e-04, 9.31324023e-05,
       7.70374354e-05, 7.01934392e-05, 5.96423461e-05, 5.39673981e-05,
       4.75224193e-05, 3.77774437e-05, 3.34696865e-05, 3.25772374e-05,
       2.85561685e-05, 2.62755502e-05, 2.42896275e-05, 2.15357951e-05,
       2.03869609e-05, 1.80888433e-05, 1.62898552e-05, 1.56320694e-05,
      

## 💾 **Save Results and Information in JSON File**

In [12]:
# Infos
nom_region = name_region
model = "GLO36V1"
name_season = season
freqs_mean = np.array(freqs_mean)
psd_mean = np.array(psd_mean)
todays_Date = datetime.now()
date_creation = todays_Date.isoformat() #ISO 8601 format
packages_versions = watermark(packages="numpy,widetrax,xarray,s3fs,pyinterp,datetime,json,platform,time")
machine_info = {
    "system": platform.system(),
    "node_name": platform.node(),
    "release": platform.release(),
    "version": platform.version(),
    "machine": platform.machine(),
    "processor": platform.processor(),
    "python_version": platform.python_version()
}

# JSON file structure
donnees = {
    "region": nom_region,
    "model": model,
    "lon_min": lon_min,
    "lon_max": lon_max,
    "lat_min": lat_min,
    "lat_max": lat_max,
    "start_date": datetime.strptime(start_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "end_date": datetime.strptime(end_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "packages_versions": packages_versions,
    "date_creation": date_creation,
    "login_creator": platform.node(),
    "Frequency": freqs_mean.tolist(),
    "Spectra": psd_mean.tolist(),
    "machine_info": machine_info
}

# Save on JSON file
with open(str(nom_region)+"_"+str(name_season)+"_"+str(model)+".json", "w", encoding="utf-8") as f:
    json.dump(donnees, f, ensure_ascii=False, indent=4)
#with open("OMIP_"+str(model)+"_"+str(nom_region)+"_"+str(name_season)+".json", "w", encoding="utf-8") as f:
  #  json.dump(donnees, f, ensure_ascii=False, indent=4)

#<region>-<period>-<data-source>.json
print("JSON file created")

JSON file created


## 📤 **Export Results to the S3 Endpoint** 

In [13]:
fs = s3fs.S3FileSystem( anon=True, endpoint_url="https://minio.lab.dive.edito.eu", use_ssl=False ) 

In [14]:
json_file = str(nom_region)+"_"+str(name_season)+"_"+str(model)+".json"
fs.put(json_file, "project-meom-ige/OMIP/")

[None]

In [17]:
fs